In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%run ../load_data.py
%run ../config_utils.py

# Test generator function

In [ ]:
batch_size=128
#filepath='/home/zhampel/insight/data/mnist/pngs'
trainpath='/home/zhampel/insight/data/cifar-10-images'
data_dir_struct = DataDirStruct(trainpath)
train_gen, valid_gen = train_img_generator(dir_struct=data_dir_struct, \
                                           batch_size=batch_size, \
                                           val_split=0.2)
print(train_gen.n)
epochs = train_gen.n / train_gen.batch_size
print(epochs, int(epochs))

In [ ]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
Y_train = np_utils.to_categorical(y_train, 10)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
datagen.fit(X_train)
datagen.fit(X_test)
train_gen = datagen.flow(x=X_train, y=Y_train, \
                         batch_size=batch_size, \
                         subset='training')
#print(train_gen.image_shape)

In [ ]:
train_gen.

In [ ]:
test_gen = test_img_generator(dir_struct=data_dir_struct, \
                              batch_size=batch_size)
print(test_gen.image_shape)
print(test_gen.num_classes)
print(test_gen.batch_size)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

np.random.seed(1671)
np.set_printoptions(threshold='nan')

In [ ]:
import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential

In [ ]:
class History(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []
        self.loss = []
        self.val_loss = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

In [ ]:
# Grab input shape and class numbers from generator
# Specifics
batch_size = test_gen.batch_size
epochs = 100
input_shape = train_gen.image_shape
num_classes = train_gen.num_classes
# History
history = History()
# Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
# Run the training
model.fit_generator(train_gen,
                    steps_per_epoch=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=valid_gen,
                    validation_steps=10,
                    class_weight=None,
                    callbacks=[history])

In [ ]:
# Evaluate score
score = model.evaluate_generator(test_gen, steps=20, max_queue_size=256)
print("Test score: {}".format(score[0]))
print("Test accuracy: {}".format(score[1]))

In [ ]:
# Summarize history
f = plt.figure(figsize=(14,5))

# Summarize accuracy history
plt.subplot(1,2,1)
plt.plot(history.acc)
plt.plot(history.val_acc)
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.subplot(1,2,2)
# Summarize loss history
plt.plot(history.loss)
plt.plot(history.val_loss)
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper right')
#plt.show()